In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"chandaninovitasari","key":"fb8cb9eeab4dd9ce4bbecea03f9dcbf9"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [3]:
!kaggle datasets download -d fatemehmehrparvar/liver-disorders

Dataset URL: https://www.kaggle.com/datasets/fatemehmehrparvar/liver-disorders
License(s): Attribution 4.0 International (CC BY 4.0)
  0% 0.00/7.63k [00:00<?, ?B/s]
100% 7.63k/7.63k [00:00<00:00, 11.1MB/s]


In [4]:
!unzip liver-disorders.zip -d liver-disorders
!ls liver-disorders

Archive:  liver-disorders.zip
  inflating: liver-disorders/Indian Liver Patient Dataset (ILPD).csv  
'Indian Liver Patient Dataset (ILPD).csv'


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

import seaborn as sns
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.models import Sequential
from keras.utils import to_categorical
from keras import callbacks

import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [6]:
df = pd.read_csv("/content/liver-disorders/Indian Liver Patient Dataset (ILPD).csv")

In [7]:
df.head()

Age  Gender    TB   DB  Alkphos  Sgpt  Sgot   TP  ALB  A/G Ratio  Selector
0   65  Female   0.7  0.1      187    16    18  6.8  3.3       0.90         1
1   62    Male  10.9  5.5      699    64   100  7.5  3.2       0.74         1
2   62    Male   7.3  4.1      490    60    68  7.0  3.3       0.89         1
3   58    Male   1.0  0.4      182    14    20  6.8  3.4       1.00         1
4   72    Male   3.9  2.0      195    27    59  7.3  2.4       0.40         1

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Age        583 non-null    int64  
 1   Gender     583 non-null    object 
 2   TB         583 non-null    float64
 3   DB         583 non-null    float64
 4   Alkphos    583 non-null    int64  
 5   Sgpt       583 non-null    int64  
 6   Sgot       583 non-null    int64  
 7   TP         583 non-null    float64
 8   ALB        583 non-null    float64
 9   A/G Ratio  579 non-null    float64
 10  Selector   583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 50.2+ KB


In [9]:
col_names = ['Gender']

for col in col_names:
    print (df[col].value_counts())

Gender
Male      441
Female    142
Name: count, dtype: int64


In [10]:
numerical = []
catgcols =[]

for col in df.columns:
  if df[col].dtype=="float64":
    numerical.append(col)
  else:
    catgcols.append(col)

for col in df.columns:
  if col in numerical:
    df[col].fillna(df[col].median(), inplace=True)
  else:
    df[col].fillna(df[col].mode()[0], inplace=True)

In [11]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in catgcols:
    df[col] = le.fit_transform(df[col])

In [12]:
import plotly.express as px
fig = px.histogram(df, x='Selector')

fig.update_layout(title_text='<b>Liver Disorders</b>', title_x=0.5)

fig.show()

In [13]:
df.drop(columns=['A/G Ratio'], inplace=True)

In [14]:
X = df.drop("Selector",axis=1)
Y = pd.get_dummies(df['Selector'])

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25)

print(f"Shape of X_train is: {X_train.shape}")
print(f"Shape of Y_train is: {Y_train.shape}\n")
print(f"Shape of X_test is: {X_test.shape}")
print(f"Shape of Y_test is: {Y_test.shape}")

Shape of X_train is: (437, 9)
Shape of Y_train is: (437, 2)

Shape of X_test is: (146, 9)
Shape of Y_test is: (146, 2)


In [16]:
input_dim = X.shape[1]

In [17]:
model = Sequential()
model.add(Dense(8, input_dim = input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(5, activation = "relu", kernel_initializer='normal'))
model.add(Dense(2, activation = 'sigmoid'))

In [18]:
model.compile(optimizer = Adam(learning_rate = 0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 80        
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 2)                 12        
                                                                 
Total params: 137 (548.00 Byte)
Trainable params: 137 (548.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
result = model.fit(X_train, Y_train, epochs = 50, batch_size = 10)

Epoch 1/50
44/44 [==============================] - 0s 4ms/step - loss: 0.5222 - accuracy: 0.7254
Epoch 2/50
44/44 [==============================] - 0s 4ms/step - loss: 0.5156 - accuracy: 0.7208
Epoch 3/50
44/44 [==============================] - 0s 5ms/step - loss: 0.5153 - accuracy: 0.7231
Epoch 4/50
44/44 [==============================] - 0s 4ms/step - loss: 0.5138 - accuracy: 0.7208
Epoch 5/50
44/44 [==============================] - 0s 6ms/step - loss: 0.5116 - accuracy: 0.7231
Epoch 6/50
44/44 [==============================] - 0s 6ms/step - loss: 0.5113 - accuracy: 0.7231
Epoch 7/50
44/44 [==============================] - 0s 8ms/step - loss: 0.5128 - accuracy: 0.7277
Epoch 8/50
44/44 [==============================] - 0s 7ms/step - loss: 0.5093 - accuracy: 0.7208
Epoch 9/50
44/44 [==============================] - 0s 7ms/step - loss: 0.5089 - accuracy: 0.7231
Epoch 10/50
44/44 [==============================] - 0s 11ms/step - loss: 0.5105 - accuracy: 0.7185
Epoch 11/50
44/44 

In [24]:
y_pred =model.predict(X_test)
y_pred = (y_pred>0.5)

5/5 [==============================] - 0s 3ms/step


In [25]:
accuracy = accuracy_score(Y_test,y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.7054794520547946


In [26]:
model.save('./liver.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning:

You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.



In [27]:
model = tf.keras.models.load_model('./liver.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open("./liver.tflite", "wb").write(tflite_model)

2612